In [24]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from tqdm import tqdm
import time
import pickle
import requests
import os
import pathlib

In [25]:
driver = webdriver.Chrome('../../chromedriver')

In [26]:
whitelist = ['minimal', 'landscape', 'abstract']
blacklist = ['phone']
WEBSITE_PAUSE_TIME = 1
IMAGE_PAUSE_TIME = 0.5
SCROLL_PAUSE_TIME = 1


In [27]:
image_urls=[]
viewed_pages = set()
image_categories={}

queue = [
    'https://wallpaperplay.com/board/4k-minimalist-wallpapers'
]

In [28]:
def scroll_to_bottom(driver):
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [29]:
driver.get('https://wallpaperplay.com/')
scroll_to_bottom(driver)
categories = driver.find_elements(By.XPATH, '//a[@class="ui fluid image"]')
for cat in categories:
    cat_url = cat.get_attribute('href')
    eval_val = 0
        
    for whitelist_word in whitelist:
        if whitelist_word in cat_url:
            eval_val += 1
    for blacklist_word in blacklist:
        if blacklist_word in cat_url:
            eval_val -= 1
    if eval_val > 0:
        queue.insert(0, cat_url)
    else:
        queue.append(cat_url)
        
print(len(queue))

5529


In [ ]:
# page_limit = 200*10
i=0
while True:
#     if i > page_limit:
#         break
    
    if len(queue) <= 0:
        break
        
    url = queue.pop(0)
    
    if url in viewed_pages:
        continue
    
    driver.get(url)
    
    time.sleep(WEBSITE_PAUSE_TIME)
    scroll_to_bottom(driver)
    time.sleep(WEBSITE_PAUSE_TIME)
    
    imgs = driver.find_elements(By.XPATH, '//img[@class="isWiden thumb ads_popup"]')
    
    image_category = url.split('/')[-1]
    
    print(url, len(imgs))
    
    for img in imgs:
        img_url = img.get_attribute('data-download')
        image_urls.append(img_url)
        
        if img_url not in image_categories:
            image_categories[img_url] = []
        image_categories[img_url].append(url)
        
        img_id = img_url.split('/')[-1]
        path = pathlib.Path('/data1', 'Raw', 'WallpaperPlay', image_category, '{}.jpg'.format(img_id))
        path.parent.mkdir(parents=True, exist_ok=True)
        
        with open(path,'wb') as f:
            f.write(requests.get(img_url).content)
            f.close()
        
        time.sleep(IMAGE_PAUSE_TIME)
        
    print(len(image_urls))
        
    related_wallpapers = driver.find_elements(By.XPATH, '//div[@class="column collection_thumb"]/a')
    
    for link in related_wallpapers:
        wp_link = link.get_attribute('href')
        eval_val = 0
        
        for whitelist_word in whitelist:
            if whitelist_word in wp_link:
                eval_val += 1
        for blacklist_word in blacklist:
            if blacklist_word in wp_link:
                eval_val -= 1
        if eval_val > 0:
            queue.insert(0, wp_link)
        else:
            queue.append(wp_link)
        
    viewed_pages.add(url)
    
    i+=1
    if (i%50) == 0:
        with open('/data1/wallpaperPlayPhotos.pkl', 'wb') as f:
            pickle.dump(image_urls, f)

        with open('/data1/wallpaperPlayPhotoCategories.pkl', 'wb') as f:
            pickle.dump(image_categories, f)
            
    time.sleep(WEBSITE_PAUSE_TIME)

https://wallpaperplay.com/board/abstract-butterfly-wallpapers 62
62
https://wallpaperplay.com/board/abstract-pattern-wallpapers 58
120
https://wallpaperplay.com/board/purple-abstract-wallpapers 73
193
https://wallpaperplay.com/board/abstract-music-wallpapers 62
255
https://wallpaperplay.com/board/trippy-landscape-wallpapers 66
321
https://wallpaperplay.com/board/winter-landscapes-wallpapers 70
391
https://wallpaperplay.com/board/abstract-skull-wallpapers 70
461
https://wallpaperplay.com/board/abstract-purple-wallpapers 69
530
https://wallpaperplay.com/board/3d-abstract-wallpapers 77
607
https://wallpaperplay.com/board/abstract-wallpapers 69
676
https://wallpaperplay.com/board/batman-minimalist-wallpapers 65
741
https://wallpaperplay.com/board/windows-landscape-wallpapers 74
815
https://wallpaperplay.com/board/abstract-dragon-wallpapers 67
882
https://wallpaperplay.com/board/abstract-hd-wallpapers 77
959
https://wallpaperplay.com/board/abstract-art-wallpapers 67
1026
https://wallpaperpl

8313
https://wallpaperplay.com/board/ghost-band-wallpapers 66
8379
https://wallpaperplay.com/board/anime-gun-wallpapers 74
8453
https://wallpaperplay.com/board/quiet-mgs-wallpapers 81
8534
https://wallpaperplay.com/board/lisa-frank-wallpapers 33
8567
https://wallpaperplay.com/board/youtube-banner-wallpapers 86
8653
https://wallpaperplay.com/board/linkedin-wallpapers 65
8718
https://wallpaperplay.com/board/4k-dark-wallpapers 55
8773
https://wallpaperplay.com/board/undertale-mettaton-wallpapers 65
8838
https://wallpaperplay.com/board/undertale-undyne-wallpapers 70
8908
https://wallpaperplay.com/board/undertale-gaster-wallpapers 53
8961
https://wallpaperplay.com/board/rwby-desktop-wallpapers 73
9034
https://wallpaperplay.com/board/grenade-wallpapers 61
9095
https://wallpaperplay.com/board/blue-lightning-wallpapers 63
9158
https://wallpaperplay.com/board/erased-anime-wallpapers 75
9233
https://wallpaperplay.com/board/rwby-yang-wallpapers 72
9305
https://wallpaperplay.com/board/noragami-ara

16117
https://wallpaperplay.com/board/black-grid-wallpapers 60
16177
https://wallpaperplay.com/board/batman-1966-wallpapers 61
16238
https://wallpaperplay.com/board/motherboard-hd-wallpapers 65
16303
https://wallpaperplay.com/board/sad-anime-wallpapers 81
16384
https://wallpaperplay.com/board/awesome-desktop-wallpapers 71
16455
https://wallpaperplay.com/board/blue-desktop-wallpapers 72
16527
https://wallpaperplay.com/board/gangsta-anime-wallpapers 76
16603
https://wallpaperplay.com/board/red-space-wallpapers 67
16670
https://wallpaperplay.com/board/hd-noragami-wallpapers 72
16742
https://wallpaperplay.com/board/epic-space-wallpapers 75
16817
https://wallpaperplay.com/board/minion-bob-wallpapers 66
16883
https://wallpaperplay.com/board/funny-iphone-wallpapers 10
16893
https://wallpaperplay.com/board/ghost-pokemon-wallpapers 64
16957
https://wallpaperplay.com/board/knights-templar-wallpapers 65
17022
https://wallpaperplay.com/board/3440x1440-hd-wallpapers 77
17099
https://wallpaperplay.c

23725
https://wallpaperplay.com/board/gold-glitter-wallpapers 45
23770
https://wallpaperplay.com/board/cool-animal-wallpapers 69
23839
https://wallpaperplay.com/board/rubiks-cube-wallpapers 54
23893
https://wallpaperplay.com/board/light-brown-wallpapers 48
23941
https://wallpaperplay.com/board/rich-gang-wallpapers 55
23996
https://wallpaperplay.com/board/cute-sloth-wallpapers 51
24047
https://wallpaperplay.com/board/watercolor-flowers-wallpapers 50
24097
https://wallpaperplay.com/board/sheepdog-police-wallpapers 42
24139
https://wallpaperplay.com/board/100-emoji-wallpapers 45
24184
https://wallpaperplay.com/board/jimmy-butler-wallpapers 71
24255
https://wallpaperplay.com/board/mm-candy-wallpapers 64
24319
https://wallpaperplay.com/board/hd-photography-wallpapers 76
24395
https://wallpaperplay.com/board/jesus-resurrection-wallpapers 65
24460
https://wallpaperplay.com/board/texture-background-wallpapers 59
24519
https://wallpaperplay.com/board/rustic-background-wallpapers 57
24576
https:

31246
https://wallpaperplay.com/board/best-mom-wallpapers 61
31307
https://wallpaperplay.com/board/film-noir-wallpapers 61
31368
https://wallpaperplay.com/board/waluigi-wallpapers 58
31426
https://wallpaperplay.com/board/autumn-pumpkin-wallpapers 52
31478
https://wallpaperplay.com/board/animated-panda-wallpapers 57
31535
https://wallpaperplay.com/board/creepy-cute-wallpapers 53
31588
https://wallpaperplay.com/board/voltron-hd-wallpapers 76
31664
https://wallpaperplay.com/board/space-alien-wallpapers 76
31740
https://wallpaperplay.com/board/shadow-trooper-wallpapers 58
31798
https://wallpaperplay.com/board/armored-core-wallpapers 59
31857
https://wallpaperplay.com/board/corsair-desktop-wallpapers 71
31928
https://wallpaperplay.com/board/space-battle-wallpapers 62
31990
https://wallpaperplay.com/board/ganondorf-wallpapers 69
32059
https://wallpaperplay.com/board/8k-space-wallpapers 29
32088
https://wallpaperplay.com/board/christmas-pc-wallpapers 68
32156
https://wallpaperplay.com/board/b

...

https://wallpaperplay.com/board/princess-crown-wallpapers 28
76849
https://wallpaperplay.com/board/lucario-wallpapers 65
76914
https://wallpaperplay.com/board/kiznaiver-wallpapers 72
76986
https://wallpaperplay.com/board/ancient-greece-wallpapers 50
77036
https://wallpaperplay.com/board/solid-gray-wallpapers 50
77086
https://wallpaperplay.com/board/sonic-riders-wallpapers 66
77152
https://wallpaperplay.com/board/nintendo-birthday-wallpapers 56
77208
https://wallpaperplay.com/board/king-diamond-wallpapers 58
77266
https://wallpaperplay.com/board/windows-nt-wallpapers 74
77340
https://wallpaperplay.com/board/artsy-wallpapers 63
77403
https://wallpaperplay.com/board/tardis-interior-wallpapers 48
77451
https://wallpaperplay.com/board/space-scenes-wallpapers 53
77504
https://wallpaperplay.com/board/ktm-logo-wallpapers 65
77569
https://wallpaperplay.com/board/1080p-christian-wallpapers 60
77629
https://wallpaperplay.com/board/msi-1920x1080-wallpapers 77
77706
https://wallpaperplay.com/board/

84267
https://wallpaperplay.com/board/shadow-lugia-wallpapers 57
84324
https://wallpaperplay.com/board/jazz-music-wallpapers 52
84376
https://wallpaperplay.com/board/scenic-desktop-wallpapers 65
84441
https://wallpaperplay.com/board/scene-girl-wallpapers 69
84510
https://wallpaperplay.com/board/vampire-knight-wallpapers 71
84581
https://wallpaperplay.com/board/drake-waterfowl-wallpapers 47
84628
https://wallpaperplay.com/board/freddy-fazbear-wallpapers 82
84710
https://wallpaperplay.com/board/marvel-iphone-wallpapers 19
84729
https://wallpaperplay.com/board/wrestling-ring-wallpapers 65
84794
https://wallpaperplay.com/board/hd-white-wallpapers 65
84859
https://wallpaperplay.com/board/awesome-hd-wallpapers 74
84933
https://wallpaperplay.com/board/lion-face-wallpapers 65
84998
https://wallpaperplay.com/board/hell-wallpapers 60
85058
https://wallpaperplay.com/board/springtime-wallpapers 59
85117
https://wallpaperplay.com/board/mexico-beach-wallpapers 48
85165
https://wallpaperplay.com/boar

91820
https://wallpaperplay.com/board/kyoto-wallpapers 55
91875
https://wallpaperplay.com/board/computer-circuit-wallpapers 64
91939
https://wallpaperplay.com/board/white-lace-wallpapers 28
91967
https://wallpaperplay.com/board/christmas-vacation-wallpapers 60
92027
https://wallpaperplay.com/board/twilight-zone-wallpapers 69
92096
https://wallpaperplay.com/board/smoking-guns-wallpapers 57
92153
https://wallpaperplay.com/board/mega-gengar-wallpapers 60
92213
https://wallpaperplay.com/board/peanuts-halloween-wallpapers 47
92260
https://wallpaperplay.com/board/charger-hellcat-wallpapers 64
92324
https://wallpaperplay.com/board/barn-owl-wallpapers 60
92384
https://wallpaperplay.com/board/seasons-wallpapers 56
92440
https://wallpaperplay.com/board/1920x1080-fallout-wallpapers 77
92517
https://wallpaperplay.com/board/comic-book-wallpapers 63
92580
https://wallpaperplay.com/board/pokemon-pokeball-wallpapers 63
92643
https://wallpaperplay.com/board/synthwave-wallpapers 72
92715
https://wallpap

99511
https://wallpaperplay.com/board/dragonfly-wallpapers 58
99569
https://wallpaperplay.com/board/god-wallpapers 38
99607
https://wallpaperplay.com/board/doom-hd-wallpapers 74
99681
https://wallpaperplay.com/board/morel-mushroom-wallpapers 56
99737
https://wallpaperplay.com/board/tech-backgrounds-wallpapers 73
99810
https://wallpaperplay.com/board/red-wallpapers 52
99862
https://wallpaperplay.com/board/nissan-logo-wallpapers 69
99931
https://wallpaperplay.com/board/73-powerstroke-wallpapers 45
99976
https://wallpaperplay.com/board/stadium-crowd-wallpapers 67
100043
https://wallpaperplay.com/board/scorpio-zodiac-wallpapers 46
100089
https://wallpaperplay.com/board/catholic-advent-wallpapers 46
100135
https://wallpaperplay.com/board/magic-eye-wallpapers 41
100176
https://wallpaperplay.com/board/fantasy-art-wallpapers 69
100245
https://wallpaperplay.com/board/tinkerbell-pictures-wallpapers 68
100313
https://wallpaperplay.com/board/fall-forest-wallpapers 65
100378
https://wallpaperplay.c

106978
https://wallpaperplay.com/board/white-phone-wallpapers 19
106997
https://wallpaperplay.com/board/xbox-controller-wallpapers 68
107065
https://wallpaperplay.com/board/cute-creepypasta-wallpapers 44


In [ ]:
with open('/data1/wallpaperPlayPhotos.pkl', 'wb') as f:
    pickle.dump(image_urls, f)

with open('/data1/wallpaperPlayPhotoCategories.pkl', 'wb') as f:
    pickle.dump(image_categories, f)